In [28]:
import warnings
warnings.filterwarnings('ignore')

import os
import sys
import gc

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# прописывем нужные нам пути
dir_project = 'git/proj_news_viz/nlp' # путь проекта

PATH = os.path.join(os.getenv('HOME'), dir_project)
sys.path.append(PATH)

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import multiprocessing as mp
from multiprocessing import Pool
num_cores = mp.cpu_count() # number of cores on your machine; 20
num_partitions = num_cores * 2 # number of partitions to split dataframe

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

---
возьмем для примера датасет [lenta.ru](https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta)

для примера взят датасет, скаченный по линку с gitHub

In [5]:
df = pd.read_csv(f'{PATH}/data/external/news_lenta.csv')
print(df.shape)
df.head(2)

(699746, 5)


,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,https://lenta.ru/news/2018/07/16/foes/


In [6]:
# удалим пустые записи и дубликаты по признаку `text`
df.dropna(inplace=True)
df.text.drop_duplicates(inplace=True)

print(f'df.shape: {df.shape}')
df.isnull().sum()

df.shape: (675697, 5)


tags     0
text     0
title    0
topic    0
url      0
dtype: int64

In [7]:
# конкатенируем признаки `title` & `text`
df['full_text'] = df.title + ' ' + df.text
df.sample(2)

,tags,text,title,topic,url,full_text
647983,Все,71-летний американец в понедельник расстрелял ...,"71-летний американец расстрелял из ""Калашников...",Мир,https://lenta.ru/news/2002/06/11/transport/,"71-летний американец расстрелял из ""Калашников..."
351072,Все,Британский производитель вооружения BAE System...,Дефицит бюджета Великобритании заставит BAE Sy...,Экономика,https://lenta.ru/news/2010/12/09/tula/,Дефицит бюджета Великобритании заставит BAE Sy...


---
препроцессим

In [33]:
from news_viz.data_processing.preprocessing_tools import clean_text, lemmatize

* clean text

In [34]:
def multiply_clean_text(data):
    data['clean_text'] = data['full_text'].apply(clean_text)
    return data

In [35]:
%%time

df = parallelize_dataframe(df, multiply_clean_text)
df = df[df.clean_text!=9999]

gc.collect()

CPU times: user 46 s, sys: 2min 14s, total: 3min
Wall time: 5min 59s


* lemmatize

In [38]:
def multiply_lemmatization(data):
    data['lemma_text'] = data['clean_text'].apply(lemmatize)
    return data

In [39]:
%%time

df = parallelize_dataframe(df, multiply_lemmatization)
df = df[df.lemma_text!=9999]
gc.collect()

CPU times: user 54.6 s, sys: 3min 58s, total: 4min 53s
Wall time: 19min 36s


In [42]:
print(df.shape)
df.sample(2)

(674580, 8)


,tags,text,title,topic,url,full_text,clean_text,lemma_text
667895,Все,"Проект индивидуальных счетов граждан, который ...",Государство разрешит гражданам не платить нало...,Экономика,https://lenta.ru/news/2001/07/26/boevik2/,Государство разрешит гражданам не платить нало...,государство разрешит гражданам не платить нало...,государство разрешить гражданин платить налог ...
317986,Все,"Власти валлийского города Портмадог, расположе...",Мост для летучих мышей обошелся валлийскому го...,Из жизни,https://lenta.ru/news/2011/09/26/micex/,Мост для летучих мышей обошелся валлийскому го...,мост для летучих мышей обошелся валлийскому го...,мост летучий мышей обойтись валлийский город т...


---

In [43]:
df[['url','lemma_text']].to_csv(f'{PATH}/data/interim/news_lenta-cleaned-lemmatize.csv.gz', compression='gzip', index=False)